In [1]:
import cv2
import os 
import numpy as np
from skimage import filters 

In [2]:
def eq1(img):   # implimentation of III B 1 of paper
  t=filters.threshold_otsu(img)  # finding threshold usign otsu algo
  flimage=np.ravel(img) # flattening the image
  im1=[]  # creating two different array based on threshold
  im2=[]
  for v in flimage:
    if(v<t):
      im1.append(v)
    else:
      im2.append(v)

  im1=np.array(im1)
  im2=np.array(im2)
  if(len(im1)==0 or len(im2)==0):
    return 0
  m1=np.mean(im1)  # finding mean, variance of the two images histogram
  m2=np.mean(im2)
  v1=np.sqrt(np.var(im1))
  v2=np.sqrt(np.var(im2))
  z2= v1**(-2) - v2**(-2)  # using the mean and variance to get the coefficent of the equation 4 of ppr
  z1= -2* ( (m1/(v1*v1)) - (m2/(v2*v2)) )
  c= (m1*m1)/(v1*v1) - (m2*m2)/(v2*v2) + 2*np.log(v1/v2)
  coeff = [z2, z1, c]
  z=np.roots(coeff)  # finding roots of the equation to get z(intersection point of two histogram)
  fz=0
  if(z[0]>0):
    fz=z[0]
  else:
    fz=z[1]
  fz=round(fz)  # taking the positive value of z storing in finalz(fz)
  sum1=0  # for integration: which is essentially sum, summing based on the ondition given
  sum2=0
  for i in im2:
    if(i<fz):
      sum1+=1
  for i in im1:
    if(i>fz):
      sum2+=1
  gamma=255  # here number of bins were 255
  lamda=sum1+sum2

  quality= (1+np.log(1+gamma*lamda))**-1  # getting the final quality using the formula
  return quality

In [3]:
def eq2(img):   # implimentation of III B 2 of paper
  _, thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)  # getting the threshold value 
  output = cv2.connectedComponentsWithStats(thresh, 4, cv2.CV_32S)  # using this function to get the connected component
  stats = output[2]   # ouput contains stats of the connected component(have 5 element for each component)
  numofcc=stats.shape[0]
  if(numofcc==0):
    return 1
  areas=[]
  for a in stats:
    areas.append(a[4])  # last component in stat is the area of the connected component
  max_val=max(areas)  # finding the max of the areas as given in equation
  mindex=areas.index(max_val)
  areas=np.array(areas)
  totsum=np.sum(areas)
  cu=max_val/totsum  # eq 8
  finalquality= cu + ((1-cu)/numofcc)  # final equation 9
  return finalquality


In [4]:
# im1="a1.png"
# im2="a2.png"
im1=input("Enter path to image 1: ")  # asking user to enter path to the images for quality comparision
im2=input("Enter path to image 2: ")
img1 = cv2.imread(im1)
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2 = cv2.imread(im2)
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
# getting the qualities using the functions defined above(4 qualities in total)
q11=eq1(img1)
q12=eq2(img1)
q21=eq1(img2)
q22=eq2(img2)

Enter path to image 1: /content/cvone.PNG
Enter path to image 2: /content/dlone.PNG


In [5]:
print("Final quality scores:",q11,q12,q21,q22)
# comparing quality scores to predict the best image
# choose alpha beta based:found that 2nd equation was doing better to find quality,therefore gave more weightage to it
alpha=0.4
beta=0.6
fscore1= alpha*q11 + beta*q12
fscore2= alpha*q21 + beta*q22
if(fscore1>fscore2):
  print("Image 1 have better quality")
elif(fscore1==fscore2):
  print("Images have similar quality")
else:
  print("Image 2 have better quality")

Final quality scores: 0.0714078888794851 0.8699067474459491 0.0801020614773078 0.8820716320716321
Image 2 have better quality
